* Reading the files
* Translating russian names to english
* Data Exploration
* Summarizing and check for patterns
* Train ARIMA on Items by Shops
* Predict Sales for each item by Shops

In [1]:
import os
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima_model import ARIMA
import datetime
from IPython.display import display
from pmdarima.arima import auto_arima #pip install scipy==1.2 --upgrade (since factorial is moved from scipy.misc to scipy.special

In [2]:
sales = pd.read_csv("./data/sales_train_v2.csv")

In [3]:
test = pd.read_csv("./data/test.csv")

In [4]:
print('\n Description of the table \n')
display(sales.describe())

print('\n Columns types in the table \n')
display(sales.dtypes)


 Description of the table 



,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03



 Columns types in the table 



date               object
date_block_num      int64
shop_id             int64
item_id             int64
item_price        float64
item_cnt_day      float64
dtype: object

Comparing items and shops in Train and Test dataframes

In [5]:
#Shops in train not in Test

diff_shop_id = set(sales['shop_id'].unique()) - set(test['shop_id'].unique())

diff_item_id = set(sales['item_id'].unique()) - set(test['item_id'].unique())

print('I would like to remove {} shops and {} items from my training SARIMA model'.format(len(diff_shop_id), len(diff_item_id)))

I would like to remove 18 shops and 17070 items from my training SARIMA model


In [6]:
train = sales[~sales['shop_id'].isin(diff_shop_id)]
train = sales[~sales['item_id'].isin(diff_item_id)]
train.shape

(1446006, 6)

Summarizing the sales by Shop and item by month

In [8]:
shop_item_by_month = train[['shop_id', 'item_id', 'date_block_num', 'item_cnt_day']].groupby(['shop_id', 'item_id', 'date_block_num']).agg({'item_cnt_day' : 'sum'}).reset_index()
shop_item_by_month = shop_item_by_month.sort_values(['shop_id', 'item_id', 'date_block_num'])

In [9]:
display(shop_item_by_month.describe())

,shop_id,item_id,date_block_num,item_cnt_day
count,704403.000000,704403.000000,704403.000000,704403.000000
mean,32.632766,10045.505090,19.540232,2.730221
std,16.543131,6192.178382,9.154805,11.426706
min,0.000000,30.000000,0.000000,-4.000000
25%,21.000000,4424.000000,13.000000,1.000000
50%,31.000000,9355.000000,21.000000,1.000000
75%,47.000000,15407.000000,27.000000,2.000000
max,59.000000,22167.000000,33.000000,2253.000000


Transforming rows to columns for shop_item_by_month

In [10]:
def rowstocols(df, key, value, join_how='outer'):
    """Given a dataframe, transform rows to 
        columns
    """
    assert type(df) is pd.DataFrame
    assert key in df.columns and value in df.columns
    assert join_how in ['outer', 'inner']
    
    fixed_vars = df.columns.difference([key, value])
    tibble = pd.DataFrame(columns=fixed_vars) # empty frame
    
    new_vars = sorted(df[key].unique())
    for c in new_vars:
        df_v = df[df[key] == c]
        del df_v[key]
        df_v = df_v.rename(columns = {value:c})       
        tibble = tibble.merge(df_v, on=list(fixed_vars), how=join_how)
    return tibble

shop_item_by_month_T = rowstocols(shop_item_by_month, 'date_block_num', 'item_cnt_day', join_how='outer').fillna(0)
gc.collect()
shop_item_by_month_T.shape

(134200, 36)

Categorize data in group based on the sales
* Shops-item with no sales for last 12 months (Closed Shops-item)
* Shops-item with no sales at all (no sale)
* Shops-item with no sales for first 24 months (New Shops-item)
* Shops-item with sales throughout (SARIMA)

In [11]:
#Closed Shops-item

shop_item_closed = shop_item_by_month_T[shop_item_by_month_T[shop_item_by_month_T.columns[25:]].sum(axis = 1) == 0]
shop_item_closed.shape

(28313, 36)

In [12]:
#Shops-items with no sales at all

shop_item_nosale = shop_item_by_month_T[shop_item_by_month_T[shop_item_by_month_T.columns[2:]].sum(axis = 1) == 0]
shop_item_nosale.shape

(71, 36)

In [13]:
#New Shope-item

shop_item_new = shop_item_by_month_T[shop_item_by_month_T[shop_item_by_month_T.columns[2:24]].sum(axis = 1) == 0] 
shop_item_new = shop_item_new[shop_item_new[shop_item_new.columns[25:]].sum(axis = 1) > 0] 
shop_item_new.shape

(55485, 36)

Fit an Arima model on new shops dataset

In [14]:
prediction = []
for i in range(len(shop_item_new)):
    cur_rec = shop_item_new[shop_item_new.columns[24:]][i:i+1].values.reshape(-1).tolist()
    at_arima = ARIMA(cur_rec, order=(1, 0, 0))
    at_arima_fit = at_arima.fit(disp=0)
    future_forecast = at_arima_fit.forecast()
    prediction.extend(future_forecast[0])
shop_item_new['34'] = prediction

In [17]:
shop_item_new.head()

,Unnamed: 0,shop_id,item_id,0,1,2,3,4,5,6,...,25,26,27,28,29,30,31,32,33,34
0,12517,15,10889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.207201
1,15344,44,14628,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.187039
2,21743,5,5362,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.092338
3,22925,29,16461,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.226190
4,24837,14,18320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.092338


In [15]:
#Shop - items to be modeled with SARIMA

shop_item_sarima = shop_item_by_month_T.copy()
shop_item_sarima = shop_item_sarima.drop(shop_item_sarima[shop_item_sarima[shop_item_sarima.columns[25:]].sum(axis = 1) == 0].index)
shop_item_sarima = shop_item_sarima.drop(shop_item_sarima[shop_item_sarima[shop_item_sarima.columns[2:24]].sum(axis = 1) == 0].index)
shop_item_sarima = shop_item_sarima.drop(shop_item_sarima[shop_item_sarima[shop_item_sarima.columns[2:]].sum(axis = 1) == 0].index)
shop_item_sarima.shape

(50401, 36)

Fit auto SARIMA for shops with consistent sales throughout

In [16]:
prediction1 = []
for i in range(len(shop_item_sarima),len(shop_item_sarima)):
    cur_rec = shop_item_sarima[shop_item_sarima.columns[2:]][i:i+1].values.reshape(-1).tolist()
    stepwise_model = auto_arima(cur_rec, start_p=0, max_p=12, start_q=0, max_q=12, start_P=0, max_P=12,start_Q=0, max_Q=12,
                            m=12, seasonal=True, d=1, D = 2, error_action='ignore', suppress_warnings=True,
                            trace=False,stepwise=True, maxiter = 15)
    stepwise_model.fit(cur_rec)
    future_forecast = stepwise_model.predict(n_periods=1)
    prediction1.append(round(future_forecast[0],0))
shop_item_sarima['34'] = prediction1

In [18]:
shop_item_sarima.head()

,Unnamed: 0,shop_id,item_id,0,1,2,3,4,5,6,...,25,26,27,28,29,30,31,32,33,34
0,586,2,33,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,-2.0
1,587,2,482,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,4.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,-3.0
2,588,2,491,1.0,0.0,0.0,0.0,1.0,2.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,596,2,1409,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,597,2,1467,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0


In [35]:
#concatenate the predictions from sarima and auto regressive model

sales_predicted = shop_item_sarima.append(shop_item_new)[['shop_id', 'item_id', '34']]
sales_predicted['34'][sales_predicted['34']< 0] = 0
sales_predicted.head()

,shop_id,item_id,34
0,2,33,0.0
1,2,482,0.0
2,2,491,3.0
3,2,1409,0.0
4,2,1467,0.0


In [48]:
submission = pd.merge(test, sales_predicted, how = 'left', on = ['shop_id', 'item_id']).fillna(0)
submission.rename(columns={'34' : 'item_cnt_month'}, inplace= True)
del submission['shop_id']
del submission['item_id']
submission.head()

,ID,item_cnt_month
0,0,2.000000
1,1,0.000000
2,2,0.860058
3,3,0.092338
4,4,0.000000


In [50]:
submission.to_csv('submission.csv', index=False)